In [1]:
# Imports
import pandas as pd
from enum import Enum
import math

In [5]:
# Function to convert rank string to its numerical value based on the ranking method
def convert_rank(rank_str, rank_enum):
    try:
        return rank_enum[rank_str].value
    except KeyError:
        return 0  # Default value if the string is not found in the Enum

# Function to calculate metrics
def calculate_metrics(group):
    top5_ranks = group.nlargest(5, 'Rank')['Rank'].reset_index(drop=True)
    average_rank = top5_ranks.mean()
    if len(top5_ranks) == 5:
        difference = top5_ranks.iloc[0] - top5_ranks.iloc[-1]
    else:
        difference = 0  # Default difference if there are not enough players
    return pd.Series([average_rank, difference], index=['Average', 'Difference'])

# Function to apply ranking method and calculate metrics
def average_top5player_teams_with_metrics(df, rank_enum):
    df['Rank'] = df['Rank'].apply(lambda x: convert_rank(x, rank_enum))
    result_df = df.groupby('Team').apply(calculate_metrics).reset_index()
    result_df = result_df.sort_values(by='Average', ascending=False)
    return result_df


In [6]:
df_original = pd.read_excel('TeamsLoL.xlsx')
combined_results = df_original[['Team']].drop_duplicates().sort_values(by='Team').reset_index(drop=True)

# Define a list of the three ranking methods
ranking_methods = [Rank_Method_One_LoL, Rank_Method_Two_LoL, Rank_Method_Three_LoL]

# Write to an Excel file with multiple sheets and print results
with pd.ExcelWriter('TeamRankMetricsOutput.xlsx', engine='openpyxl') as writer:
    for rank_enum in ranking_methods:
        df = pd.read_excel('TeamsOutputSheet.xlsx')  # Reload the DataFrame
        result_df = average_top5player_teams_with_metrics(df, rank_enum)
        # Write to a separate sheet in the Excel workbook
        result_df.to_excel(writer, sheet_name=rank_enum.__name__, index=False)
        # Print the result for the current ranking method
        print(f"Results using {rank_enum.__name__}:\n")
        print(result_df)
        print("\n" + "-"*60 + "\n")  # Separator for readability

Results using Rank_Method_One_LoL:

                         Team  Average  Difference
28   Zephyr Loch Ness Monster   5880.0       600.0
10          Grizzled Gamblers   5660.0      1100.0
18              Lars de Vries   5560.0      1600.0
2                  Blitzfrank   5160.0      2000.0
7           Duivelsei Academy   5120.0      2700.0
9              Green Tea Love   4720.0      1400.0
8         Esports Team Twente   4600.0      3000.0
27              Zephyr Kratos   4140.0      4200.0
20       Peritan the Platypus   3880.0      1400.0
17           Iron Bucket Boys   3880.0      2700.0
22           TSEA Link Reborn   3700.0      3600.0
29            Zephyr Odysseus   3540.0      4000.0
25               Zephyr Hydra   3520.0      1000.0
31               Zephyr Shiva   3400.0      1000.0
19                 Padvinders   3320.0      2200.0
13              Ignite Orange   3280.0      1700.0
6        Dirty Little Animals   3200.0      2200.0
1                  BS Scaling   2960.0      31